In [30]:
#Hendra Rizki Fatoni
#Final Task_ID/X Partners_Data Scientist
#Data Preparation_Modelling_Evaluation

import pandas as pd

# Memuat dataset
file_path = 'loan_data_2007_2014.csv'
data = pd.read_csv(file_path)

# Mendapatkan jumlah baris dan kolom
num_rows, num_columns = data.shape
print(f'Jumlah baris: {num_rows}', 'baris')
print(f'Jumlah kolom: {num_columns}', 'kolom')

# Mendefinisikan dictionary untuk mengganti nama kolom
nama_kolom_baru = {
    "loan_status": "Status_Pinjaman",
}

# Mengganti nama kolom
data.rename(columns=nama_kolom_baru, inplace=True)
# Mendapatkan nama kolom
columns = data.columns.tolist()

C:\Users\acer\AppData\Local\Temp\ipykernel_26232\493728247.py:5: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


Jumlah baris: 466285 baris
Jumlah kolom: 75 kolom


In [31]:
# Menghapus kolom yang seluruhnya kosong
data_cleaned = data.dropna(axis=1, how='all')
data=data_cleaned
# Identifikasi kolom dengan nilai yang hilang
missing_values = data.isnull().sum()

# Tangani nilai yang hilang
# Imputasi nilai yang hilang dengan mean untuk fitur numerik
for col in data.select_dtypes(include=['int64', 'float64']).columns:
    data[col].fillna(data[col].mean(), inplace=True)

# Imputasi nilai yang hilang dengan modus untuk fitur kategorikal
for col in data.select_dtypes(include=['object']).columns:
    data[col].fillna(data[col].mode()[0], inplace=True)
print("sukses menghapus kolom yang tidak ada nilainya")


C:\Users\acer\AppData\Local\Temp\ipykernel_26232\958471311.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mean(), inplace=True)
C:\Users\acer\AppData\Local\Temp\ipykernel_26232\958471311.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

sukses menghapus kolom yang tidak ada nilainya


In [32]:

# Mendapatkan jumlah baris dan kolom
num_rows, num_columns = data.shape
print(f'Jumlah baris: {num_rows}', 'baris')
print(f'Jumlah kolom: {num_columns}', 'kolom')


Jumlah baris: 466285 baris
Jumlah kolom: 58 kolom


In [33]:
def detect_outliers_iqr(df, columns):
    outliers = {}
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers[col] = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    return outliers

# Kolom numerik yang akan diperiksa untuk outlier
# Mendefinisikan kolom numerik
numerik_columns = data.select_dtypes(include=['float64', 'int64']).columns

# Mengidentifikasi outlier
outliers = detect_outliers_iqr(data, numerik_columns)
outliers


def remove_outliers_iqr(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR


    return df
# Menghapus outlier
data_cleaned = remove_outliers_iqr(data, numerik_columns)

def replace_outliers_iqr(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df.loc[df[col] < lower_bound, col] = lower_bound
        df.loc[df[col] > upper_bound, col] = upper_bound
    return df

# Mengganti outlier
data_transformed = replace_outliers_iqr(data_cleaned, numerik_columns)
data=data_transformed
print("Sukses mengganti outlier")


Sukses mengganti outlier


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, classification_report
from sklearn.metrics import roc_auc_score


# Pisahkan fitur dan target
# Mendefinisikan kolom numerik
numerical_columns = data.select_dtypes(include=['float64', 'int64']).columns
X = data[numerical_columns]  # Ganti 'loan_status' dengan nama kolom target Anda

from sklearn.preprocessing import LabelEncoder
# Langkah 3: Menggunakan Label Encoding untuk kolom target 'loan_status'
label_encoder = LabelEncoder()
data['Status_Pinjaman'] = label_encoder.fit_transform(data['Status_Pinjaman'])
y = data['Status_Pinjaman']  # nama kolom target

# Inisialisasi StandardScaler
scaler = StandardScaler()

# Melakukan scaling pada fitur-fitur dalam X
X_scaled = scaler.fit_transform(X)

# Bagi data yang telah di-scale menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

#solver='lbfgs', max_iter=10000, verbose=1
# Inisialisasi model
logreg = LogisticRegression()
print('Inisialisasi model LogisticRegression')
dtree = DecisionTreeClassifier()
print('Inisialisasi model DecisionTreeClassifier')
rf = RandomForestClassifier()
print('Inisialisasi model RandomForestClassifier')

# Melatih model
logreg.fit(X_train, y_train)
print('sukses Melatih model logreg')
dtree.fit(X_train, y_train)
print('sukses Melatih model dtree')
rf.fit(X_train, y_train)
print('sukses Melatih model Random forest')

# Prediksi dengan set pelatihan
y_train_pred_logreg = logreg.predict(X_train)
y_train_pred_dtree = dtree.predict(X_train)
y_train_pred_rf = rf.predict(X_train)

# Prediksi dengan set pengujian
y_pred_logreg = logreg.predict(X_test)
y_pred_dtree = dtree.predict(X_test)
y_pred_rf = rf.predict(X_test)


# Evaluasi model
print("Logistic Regression:")
print("Training Accuracy:", accuracy_score(y_train, y_train_pred_logreg))
print("Testing Accuracy:", accuracy_score(y_test, y_pred_logreg))
print("AUC-ROC:", roc_auc_score(y_test, logreg.predict_proba(X_test), multi_class='ovr'))
print("F1 Score:", f1_score(y_test, y_pred_logreg, average='weighted'))
print(classification_report(y_test, y_pred_logreg))
print("===============================================")

print("\nDecision Tree:")
print("Training Accuracy:", accuracy_score(y_train, y_train_pred_dtree))
print("Testing Accuracy:", accuracy_score(y_test,  y_pred_dtree))
print("AUC-ROC:", roc_auc_score(y_test, dtree.predict_proba(X_test), multi_class='ovr'))
print("F1 Score:", f1_score(y_test,  y_pred_dtree, average='weighted'))
print(classification_report(y_test,  y_pred_dtree))

print("\nRandom Forest:")
print("Training Accuracy:", accuracy_score(y_train, y_train_pred_rf))
print("Testing Accuracy:", accuracy_score(y_test, y_pred_rf))
print("AUC-ROC:", roc_auc_score(y_test, rf.predict_proba(X_test), multi_class='ovr'))
print("F1 Score:", f1_score(y_test, y_pred_rf, average='weighted'))
print(classification_report(y_test, y_pred_rf))



Inisialisasi model LogisticRegression
Inisialisasi model DecisionTreeClassifier
Inisialisasi model RandomForestClassifier


C:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


sukses Melatih model logreg
sukses Melatih model dtree
sukses Melatih model Random forest
Logistic Regression:
Training Accuracy: 0.9624130092111048
Testing Accuracy: 0.9619331524711282
AUC-ROC: 0.9306602822011852
F1 Score: 0.9478445902877134


C:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      8469
           1       0.95      1.00      0.97     44894
           2       0.33      0.02      0.03       173
           3       0.57      0.20      0.29       148
           4       0.54      0.18      0.27       377
           5       0.98      1.00      0.99     36906
           6       0.00      0.00      0.00       653
           7       0.00      0.00      0.00       237
           8       0.00      0.00      0.00      1400

    accuracy                           0.96     93257
   macro avg       0.48      0.37      0.39     93257
weighted avg       0.94      0.96      0.95     93257


Decision Tree:
Training Accuracy: 1.0
Testing Accuracy: 0.9597349260645314
AUC-ROC: 0.7936873202379705
F1 Score: 0.9600858738612024
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      8469
           1       0.96      0.96      0.96     44894

C:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif